# 10-714 Final Project

In this final project I will implement DLRM model (https://arxiv.org/abs/1906.00091) and training it on the Criteo 1TB Click Logs Dataset (https://labs.criteo.com/2013/12/download-terabyte-click-logs/).

As a contribution to the needle framework, I will be implementing (hopefully in a reusable manner) the following building blocks:

- Binary Cross-Entropoy loss
- hashing trick that would optimize memory footprint of the `Embedding` layer
- data parallel distributed training using ray.io

In [1]:
# # Code to set up the assignment
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/
# !mkdir -p 10714
# %cd /content/drive/MyDrive/10714
# !git clone http://github.com/chaos-ad/dlsyscourse-final.git
# %cd /content/drive/MyDrive/10714/final

# !pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
# !pip3 install pybind11

In [2]:
%pwd

'/home/ec2-user/SageMaker/code/dlsyscourse-homework/final/notebooks'

In [4]:
!pip install --quiet -r ../requirements.txt

In [5]:
!cd .. && make

-- Found pybind11: /home/ec2-user/SageMaker/.cs/conda/envs/codeserver_py39/lib/python3.9/site-packages/pybind11/include (found version "2.10.3")
-- Found cuda, building cuda backend
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

-- Configuring done
-- Generating done
-- Build files have been written to: /home/ec2-user/SageMaker/code/dlsyscourse-homework/final/build
make[1]: Entering directory `/home/ec2-user/SageMaker/code/dlsyscourse-homework/final/build'
make[2]: Entering directory `/home/ec2-user/SageMaker/code/dlsyscourse-homework/final/build'
make[3]: Entering directory `/home/ec2-user/SageMaker/code/dlsyscourse-homework/final/build'
make[3]: Leaving directory `/home/ec2-user/SageMaker/code/dlsyscourse-homework/final/build'
[ 50%] Built target ndarray_backend_cpu
make[3]: Entering directory `/home/ec2-user/SageMaker/code/dlsyscourse-homework/final/build'
make[3]: Leaving directory `/h

In [6]:
import os
import sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../python"))

In [7]:
import dotenv
_ = dotenv.load_dotenv(dotenv_path="../conf/dev.env")

In [8]:
import apps.etl
import apps.utils.aws
import apps.utils.common

## Hot code reloading, useful during dev:
%load_ext autoreload
%autoreload 1
%aimport apps.etl
%aimport apps.utils.common
%aimport apps.utils.aws.s3
%aimport apps.utils.aws.athena

In [9]:
import logging
logger = logging.getLogger("notebooks.final")
apps.utils.common.setup_logging(config_file="../conf/logging.yml")

### Download criteo 1TB dataset into S3

Run me once to download data to S3 (bucket & prefix are controlled via conf/dev.env, and postfix is an argument defaulting to "criteo/raw")

In [ ]:
apps.etl.import_criteo_dataset()

Convert data into parquet format, so that column-wise operations will be much faster

In [ ]:
apps.etl.init_parquet_athena_table()

In [ ]:
# apps.etl.parse_criteo_dataset()

Some parts of the ETL are omitted here due to lack of time. But essentially for each feature we build a lookup dictionary sorted by frequency using Athena query. Next, we join it back to the sparse features, and use their indices in a dict instead (similarly to what we have done for PTB in HW4).

In [10]:
import tests.debug

/home/ec2-user/SageMaker/.cs/conda/envs/codeserver_py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-01-10 02:06:58,911 - torch.distributed.nn.jit.instantiator - INFO - Created a temporary directory at /tmp/tmp90netoa8
2023-01-10 02:06:58,912 - torch.distributed.nn.jit.instantiator - INFO - Writing /tmp/tmp90netoa8/_remote_module_non_scriptable.py


In [ ]:
tests.debug.run_torch()

In [11]:
tests.debug.run_needle()

2023-01-10 02:07:31,188 - tests.debug - DEBUG - TRAIN[epoch_id=1] on batch_id=1 of size 16384...


MemoryError: std::bad_alloc